In [14]:
import sys
import heapq
from collections import deque


class Problem(object):
    def __init__(self, start, goal):
        """Initializes the 8 puzzle board with start and goal states."""
        # using tuple because it is hashable
        self.start = start
        self.goal = goal

    def actions(self, state):
        """
        Returns the list of actions possible in the given state.

        We define action in terms of the movement of the blank tile, i.e. the
        blank tile can move either left, right, up or down.
        """
        # find index of blank tile, i.e. 0
        ix = state.index(0)
        actions = ['left', 'right', 'up', 'down']
        if ix % 3 == 0:
            actions.remove('left')
        if ix < 3:
            actions.remove('up')
        if ix % 3 == 2:
            actions.remove('right')
        if ix > 5:
            actions.remove('down')
        return actions

    def is_goal_state(self, state):
        return self.goal == state

    def result(self, state, action):
        """Returns the new state when the action is applied to a state."""
        change = 0
        change = {'left': -1, 'right': 1, 'up': -3, 'down': 3}
        blank_ix = state.index(0)
        adjacent_ix = blank_ix + change[action]

        # swap
        new_state = list(state)  # tuple is immutable
        new_state[blank_ix], new_state[adjacent_ix] = new_state[adjacent_ix], new_state[blank_ix]

        return tuple(new_state)

    def h(self, state, func=None):
        """Returns the h estimate for the given state when a heuristic function is
        provided. Default heuristic is number of tiles at incorrect location"""
        if func is None:  # number of tiles at incorrect location
            return sum([1 for i in range(len(state)) if state[i] != self.goal[i]])
        else:
            return func(state, self.goal)


class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        self.f = path_cost
        if parent:
            self.depth = self.parent.depth + 1

    def child_node(self, problem, action):
        """Compute child node given parent node."""
        new_state = problem.result(self.state, action)
        path_cost = self.path_cost + 1
        return Node(new_state, self, action, path_cost)

    def __repr__(self):
        return '{}\t{}'.format(self.state, self.action)

    def __hash__(self):
        return hash(self.state)

    def __eq__(self, other):
        return self.state == other.state

    def __lt__(self, other):
        return self.f < other.f

### Uninformed search: BFS

In [15]:
def breadth_first_search(problem):
    """Breadth first search."""
    node = Node(problem.start)
    if problem.is_goal_state(node.state):
        return solution(node)
    frontier = deque()
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.popleft()
        explored.add(node.state)
        if node.depth <= 10:
            for action in problem.actions(node.state):
                child = node.child_node(problem, action)
                if child.state not in explored and child not in frontier:
                    if problem.is_goal_state(child.state):
                        print('Number of states enqueued: {}'.format(len(explored)))
                        return solution(child)
                    frontier.append(child)  # inserts to right
    return 'failure'

### Iterative deepening search

In [16]:
def recursive_dls(node, problem, limit):
    """Recursive helper function for depth limited search."""
    if problem.is_goal_state(node.state):
        return solution(node)
    elif limit == 0:
        return 'cutoff'
    else:
        cutoff_occured = False
        for action in problem.actions(node.state):
            child = node.child_node(problem, action)
            result = recursive_dls(child, problem, limit - 1)
            if result == 'cutoff':
                cutoff_occured = True
            elif result != 'failure':
                return result
        if cutoff_occured:
            return 'cutoff'
        else:
            return 'failure'


def depth_limited_search(problem, limit):
    node = Node(problem.start)
    return recursive_dls(node, problem, limit)


def iterative_deepening_search(problem):
    for depth in range(10):
        result = depth_limited_search(problem, depth)
        if result != 'cutoff':
            return result

### Informed Search: A* using heuristics

In [17]:
def astar_search(problem, heuristic=None):
    """A* search using a heuristic function."""
    node = Node(problem.start)
    node.path_cost = 0 + problem.h(node.state, heuristic)
    if problem.is_goal_state(node.state):
        return solution(node)
    frontier = []  # priority queue that also supports lookup
    heapq.heappush(frontier, node)
    explored = set()
    while frontier:
        node = heapq.heappop(frontier)
        if problem.is_goal_state(node.state):
            print('Number of states enqueued: {}'.format(len(explored)))
            return solution(node)
        explored.add(node.state)
        if node.depth <= 10:
            for action in problem.actions(node.state):
                child = node.child_node(problem, action)
                child.f = child.path_cost + problem.h(child.state, heuristic)
                if child.state not in explored and child not in frontier:
                    heapq.heappush(frontier, child)
                elif child in frontier:
                    # get that node
                    for el in frontier:
                        if el == child and child.f < el.f:  # same state, different path
                            frontier.remove(el)
                            heapq.heappush(frontier, child)
    return 'failure'

#### Heuristic Function: Manhattan Distance

In [18]:
def solution(node):
    """Returns the path, i.e. a sequence of nodes from start to goal state."""
    res = []
    while node.parent:
        res.append(node)
        node = node.parent
    res.append(node)
    return res[::-1]


def get_2d_index(i):
    if i <= 2:
        return 0, i
    elif i <= 5:
        return 1, i - 3
    else:
        return 2, i - 6


def dist(ix1, ix2):
    i1, j1 = get_2d_index(ix1)
    i2, j2 = get_2d_index(ix2)
    return abs(i2 - i1) + abs(j2 - j1)


def manhattan_distance(state, goal):
    res = 0
    for ix1, el in enumerate(state):
        ix2 = goal.index(el)
        res += dist(ix1, ix2)
    return res

### Tests

In [19]:
start = (2,4,3,1,5,6,7,8,0)
goal = (1,2,3,4,5,6,7,8,0)

In [20]:
puzzle = Problem(start, goal)
breadth_first_search(puzzle)

Number of states enqueued: 117


[(2, 4, 3, 1, 5, 6, 7, 8, 0)	None,
 (2, 4, 3, 1, 5, 6, 7, 0, 8)	left,
 (2, 4, 3, 1, 0, 6, 7, 5, 8)	up,
 (2, 0, 3, 1, 4, 6, 7, 5, 8)	up,
 (0, 2, 3, 1, 4, 6, 7, 5, 8)	left,
 (1, 2, 3, 0, 4, 6, 7, 5, 8)	down,
 (1, 2, 3, 4, 0, 6, 7, 5, 8)	right,
 (1, 2, 3, 4, 5, 6, 7, 0, 8)	down,
 (1, 2, 3, 4, 5, 6, 7, 8, 0)	right]

In [22]:
iterative_deepening_search(puzzle)

[(2, 4, 3, 1, 5, 6, 7, 8, 0)	None,
 (2, 4, 3, 1, 5, 6, 7, 0, 8)	left,
 (2, 4, 3, 1, 0, 6, 7, 5, 8)	up,
 (2, 0, 3, 1, 4, 6, 7, 5, 8)	up,
 (0, 2, 3, 1, 4, 6, 7, 5, 8)	left,
 (1, 2, 3, 0, 4, 6, 7, 5, 8)	down,
 (1, 2, 3, 4, 0, 6, 7, 5, 8)	right,
 (1, 2, 3, 4, 5, 6, 7, 0, 8)	down,
 (1, 2, 3, 4, 5, 6, 7, 8, 0)	right]

In [21]:
astar_search(puzzle, None)

Number of states enqueued: 21


[(2, 4, 3, 1, 5, 6, 7, 8, 0)	None,
 (2, 4, 3, 1, 5, 0, 7, 8, 6)	up,
 (2, 4, 3, 1, 0, 5, 7, 8, 6)	left,
 (2, 0, 3, 1, 4, 5, 7, 8, 6)	up,
 (0, 2, 3, 1, 4, 5, 7, 8, 6)	left,
 (1, 2, 3, 0, 4, 5, 7, 8, 6)	down,
 (1, 2, 3, 4, 0, 5, 7, 8, 6)	right,
 (1, 2, 3, 4, 5, 0, 7, 8, 6)	right,
 (1, 2, 3, 4, 5, 6, 7, 8, 0)	down]

In [23]:
astar_search(puzzle, manhattan_distance)

Number of states enqueued: 17


[(2, 4, 3, 1, 5, 6, 7, 8, 0)	None,
 (2, 4, 3, 1, 5, 0, 7, 8, 6)	up,
 (2, 4, 3, 1, 0, 5, 7, 8, 6)	left,
 (2, 0, 3, 1, 4, 5, 7, 8, 6)	up,
 (0, 2, 3, 1, 4, 5, 7, 8, 6)	left,
 (1, 2, 3, 0, 4, 5, 7, 8, 6)	down,
 (1, 2, 3, 4, 0, 5, 7, 8, 6)	right,
 (1, 2, 3, 4, 5, 0, 7, 8, 6)	right,
 (1, 2, 3, 4, 5, 6, 7, 8, 0)	down]